In [1]:
from tensorflow.keras.models import Model # Импортируем модели keras: Model
from tensorflow.keras.layers import Input, Conv2DTranspose, concatenate, Activation, Add, MaxPooling2D, Conv2D, BatchNormalization # Импортируем стандартные слои keras
from tensorflow.keras import backend as K # Импортируем модуль backend keras'а
from tensorflow.keras.optimizers import Adam # Импортируем оптимизатор Adam
from tensorflow.keras import utils # Импортируем модуль utils библиотеки tensorflow.keras для получения OHE-представления
import tensorflow as tf
from google.colab import files # Импортируем Модуль files для работы с файлами
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
from tensorflow.keras.preprocessing import image # Импортируем модуль image для работы с изображениями
import numpy as np # Импортируем библиотеку numpy
from sklearn.model_selection import train_test_split
import time
import random
import shutil
import os # Импортируем библиотеку os для раоты с фаловой системой
from PIL import Image # импортируем модель Image для работы с изображениями
import seaborn as sns
from tensorflow.keras.models import load_model, save_model
import pickle

import numba as nb
import cv2 as cv
from PIL import Image
from numpy.lib.stride_tricks import as_strided
sns.set_style('darkgrid')

In [2]:
from google.colab import drive # Подключаем гугл-диск
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Глобальные параметры
num_classes = 2 # Задаем количество классов на изображении
directory = '/content/drive/My Drive/segmentation/' # Указываем путь к обучающей выборке с оригинальными изображения

In [4]:
def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv.getPerspectiveTransform(rect, dst)
	warped = cv.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped

def get_coordinates_logic(img, factor = 100):
  color = (384, 256, 384)
  width = img.shape[1]
  height = img.shape[0]
  x_scale = width - factor + 1
  y_scale = height - factor + 1

  try:
    temp_img = img[0:factor, 0:factor, :]
    w1 = min(np.array(np.where(np.all(temp_img != color, axis=-1)))[1]) #min - minimum value along axis/max - maximum value along axis
    h1 = min(np.array(np.where(np.all(temp_img != color, axis=-1)))[0]) #[0] - target vertical axis (height)/ [1] - target horizontal axis (width)
    temp_img = img[0:factor, -factor:, :]
    w2 = max(np.array(np.where(np.all(temp_img != color, axis=-1)))[1]) + x_scale
    h2 = min(np.array(np.where(np.all(temp_img != color, axis=-1)))[0])
    temp_img = img[-factor:, -factor:, :]
    w3 = max(np.array(np.where(np.all(temp_img != color, axis=-1)))[1]) + x_scale
    h3 = max(np.array(np.where(np.all(temp_img != color, axis=-1)))[0]) + y_scale
    temp_img = img[-factor:, 0:factor, :]
    w4 = min(np.array(np.where(np.all(temp_img != color, axis=-1)))[1])
    h4 = max(np.array(np.where(np.all(temp_img != color, axis=-1)))[0]) + y_scale
  except:
    w1, w2, w3, w4, h1, h2, h3, h4 = get_coordinates_logic(img, factor = factor + 50)
  
  return w1, w2, w3, w4, h1, h2, h3, h4

def process_img(img):

  img = img.copy()
  height = img.shape[0]
  width = img.shape[1]

  itemindex = np.array(np.where(np.all(img == (384, 256, 384), axis=-1))) #Set pixel values to something outside of the uint8 range
                                                                          #(to avoid interfering with any pixels that might be in the image)
  minH = min(itemindex[0])  #Height to crop (top)
  maxH = max(itemindex[0])  #Height to crop (bottom)
  minW = min(itemindex[1])  #Width to crop (left)
  maxW = max(itemindex[1])  #Width to crop (right)

  pts = np.int32([[minW,minH], [maxW,minH], [minW,maxH], [maxW,maxH]])
  img_cropped = img[minH:maxH, minW:maxW]
  w1, w2, w3, w4, h1, h2, h3, h4 = get_coordinates_logic(img_cropped)

  img_cropped[np.all(img_cropped == (384, 256, 384), axis = -1)] = (255, 255, 255)
  img_cropped = img_cropped.astype(np.uint8) #Change back to uint8 datatype
  result = four_point_transform(img_cropped, np.array([[w1, h1], [w2, h2], [w3, h3], [w4, h4]])) #Four-point transform (trapezoid to rectangle)
  return result

def index2color(segment_img, actual_img, color = (128, 0, 128), final_mode = False):
  new_img = np.zeros_like(actual_img).astype(np.uint16)
  
  segment_img = cv.resize(segment_img, (actual_img.shape[1], actual_img.shape[0]), interpolation = cv.INTER_LANCZOS4)
  segment_img = np.round(segment_img)

  zero_index = np.where(np.all(segment_img == (1, 0), axis=-1))
  ones_index = np.where(np.all(segment_img == (0, 1), axis=-1))
  
  new_img[zero_index] = (384, 256, 384)
  new_img[ones_index] = actual_img[ones_index]
  
  if final_mode:
    new_img = process_img(new_img)
  return new_img

In [5]:
img_width = 512
img_height = 768

In [6]:
def get_model_from_file():
  def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.) # Возвращаем площадь пересечения деленную на площадь объединения двух областей


  model = load_model(directory + 'weightsUS/acc09811.h5')
  model.compile(optimizer=Adam(lr = 0.0001),
                loss='categorical_crossentropy',
                metrics=[dice_coef])
    
  return model

In [7]:
def predict_image(img_path, model):
  img = np.array(Image.open(img_path)) #Открываем картинку с входного пути
  
  actual_height = img.shape[0]         #Записываем высоту оригинальной картинки
  actual_width = img.shape[1]          #Записываем шырину оригинальной картинки

  width = 512                          #Ширина которая будет подаваться в НС
  height = 768                         #Высота которая будет подаваться в НС
  input_img = cv.resize(img, (width, height), interpolation = cv.INTER_LANCZOS4) #Делаем Reshape для входа в НС
  actual_img = cv.resize(img, (width * 2, height * 2), interpolation = cv.INTER_LANCZOS4) #Делаем Reshape для настоящей картинки
  input_img = np.expand_dims(input_img, axis = 0) #Добавляем ось для НС
  pred_image = model.predict(input_img/255).reshape((height, width, num_classes)) #Получаем результат с НС и делаем обратный предикт
  
  pred = index2color(pred_image, actual_img, color = (255, 255, 255), final_mode = True).astype(np.uint8) #Обрабатываем данные с сегментации
  pred = cv.resize(pred, (actual_width, actual_height), interpolation = cv.INTER_LANCZOS4) #Делаем reshape обратно

  pred = Image.fromarray(pred) #Переводим с numpy массива в картинку
  return pred #Возвращаем картинку

In [8]:
model_best = get_model_from_file()
predict_image(directory + 'orig/дог200.jpg', model = model_best)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i, filename in enumerate(os.listdir(directory + 'orig/')):
  pred = predict_image(directory + f'orig/{filename}', model = model_best)
  plt.figure(figsize = (20, 20))
  print(str(i) + '_' * 50)
  plt.imshow(np.array(pred))
  plt.show()